In [ ]:
import sys
import stlearn as st
st.settings.set_figure_params(dpi=300)
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sys
file = Path("../stimage").resolve()
parent= file.parent
sys.path.append(str(parent))
from PIL import Image
from stimage._utils import gene_plot, Read10X, ReadOldST, tiling
from stimage._model import CNN_NB_multiple_genes
from stimage._data_generator import DataGenerator
import tensorflow as tf
import seaborn as sns
sns.set_style("white")
import matplotlib.pyplot as plt
from scipy import stats
from anndata import read_h5ad
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
from splot.esda import moran_scatterplot, lisa_cluster
from esda.moran import Moran, Moran_Local
from esda.moran import Moran_BV, Moran_Local_BV
from splot.esda import plot_moran_bv_simulation, plot_moran_bv, plot_local_autocorrelation

In [ ]:
from scipy import stats

def plot_correlation(df, attr_1, attr_2):
    r = stats.pearsonr(df[attr_1], 
                       df[attr_2])[0] **2

    g = sns.lmplot(data=df,
        x=attr_1, y=attr_2,
        height=5, legend=True
    )
    # g.set(ylim=(0, 360), xlim=(0,360))

    g.set_axis_labels(attr_1, attr_2)
    plt.annotate(r'$R^2:{0:.2f}$'.format(r),
                (max(df[attr_1])*0.9, max(df[attr_2])*0.9))
    return g


def calculate_correlation(attr_1, attr_2):
    r = stats.pearsonr(attr_1, 
                       attr_2)[0]
    return r

def calculate_correlation_2(attr_1, attr_2):
    r = stats.spearmanr(attr_1, 
                       attr_2)[0]
    return r

In [ ]:
BASE_PATH = Path("/scratch/imb/Xiao/Q1851/Xiao/STimage/experiments/cnn_base")

In [ ]:
BASE_PATH = Path("/scratch/imb/Xiao/Q1851/Xiao/STimage/experiments/cnn_base_2000_hvg")

In [ ]:
cnn_bases = ["inceptionv3_ft", "densenet121_ft", "mobilenetv2_ft", "resnet50_ft", "vgg16_ft", "xception_ft"]

In [ ]:
gene_list=["SLITRK6", "PGM5", "LINC00645", 
           "TTLL12", "COX6C", "CPB1",
           "KRT5", "MALAT1"]
gene_list

In [ ]:
cnn_base =  cnn_bases[0]

In [ ]:
test_dataset = read_h5ad("/scratch/imb/Xiao/Q1851/Xiao/STimage_dataset/breast_cancer_10x_visium/dataset_2/test_adata.h5ad") 

In [ ]:
pred_adata = read_h5ad(BASE_PATH/cnn_base / "prediction.h5ad") 

In [ ]:
pred_adata.obs

In [ ]:
df = pd.DataFrame()
for cnn_base in cnn_bases:
    pred_adata = read_h5ad(BASE_PATH/cnn_base / "prediction.h5ad")
    for gene in pred_adata.var_names:
        cor_val = calculate_correlation(pred_adata.to_df().loc[:,gene], test_dataset.to_df().loc[:,gene])
        df = df.append(pd.Series([cnn_base, gene, cor_val], index=["cnn_base", "gene", "pearson_cor"]),
                  ignore_index=True)

In [ ]:
df.groupby("cnn_base").std()

In [ ]:
df.groupby("cnn_base").mean()

In [ ]:
df = pd.DataFrame()
for cnn_base in cnn_bases:
    pred_adata = read_h5ad(BASE_PATH/cnn_base / "prediction.h5ad")
    for gene in pred_adata.var_names:
        cor_val = calculate_correlation_2(pred_adata.to_df().loc[:,gene], test_dataset.to_df().loc[:,gene])
        df = df.append(pd.Series([cnn_base, gene, cor_val], index=["cnn_base", "gene", "pearson_cor"]),
                  ignore_index=True)

In [ ]:
df.groupby("cnn_base").std()

In [ ]:
df.groupby("cnn_base").mean()

In [ ]:
df = pd.DataFrame()
for cnn_base in cnn_bases:
    pred_adata = read_h5ad(BASE_PATH/cnn_base / "prediction.h5ad")
    for gene in pred_adata.var_names:
        test_dataset.obsm["gpd"] = gpd.GeoDataFrame(test_dataset.obs,
                                             geometry=gpd.points_from_xy(
                                                 test_dataset.obs.imagecol, 
                                                 test_dataset.obs.imagerow))
        x = test_dataset.to_df()[gene].values
        y = pred_adata.to_df()[gene].values
        w = Queen.from_dataframe(test_dataset.obsm["gpd"])
        moran = Moran(y,w)
        moran_bv = Moran_BV(y, x, w)
        moran_loc = Moran_Local(y, w)
        moran_loc_bv = Moran_Local_BV(y, x, w)
        df = df.append(pd.Series([cnn_base, gene, moran_bv.I], index=["cnn_base", "gene", "pearson_cor"]),
                  ignore_index=True)

In [ ]:
df.groupby("cnn_base").std()

In [ ]:
df.groupby("cnn_base").mean()

In [ ]:
BASE_PATH = Path("/scratch/imb/Xiao/Q1851/Xiao/STimage/experiments/cnn_base_2000_hvg/resnet50_ft")

In [ ]:
BASE_PATH = Path("/scratch/imb/Xiao/Q1851/Xiao/STimage/experiments/cnn_base/resnet50_ft")

In [ ]:
pred_adata = read_h5ad(BASE_PATH / "prediction.h5ad")
test_dataset = read_h5ad("/scratch/imb/Xiao/Q1851/Xiao/STimage_dataset/breast_cancer_10x_visium/dataset_2_HVG2000/test_adata.h5ad")


In [ ]:
pred_adata = read_h5ad(BASE_PATH / "prediction.h5ad")
test_dataset = read_h5ad("/scratch/imb/Xiao/Q1851/Xiao/STimage_dataset/breast_cancer_10x_visium/dataset_2/test_adata.h5ad")



In [ ]:
adata_list=[]
for adata in [
    pred_adata,
    test_dataset
]:
    
    adata_ = adata.copy()
    st.em.run_pca(adata_,n_comps=7)
    adata_.uns["spatial"] = {}
    adata_.uns["spatial"][pred_adata.obs.library_id.unique()[0]] = adata.uns["spatial"].pop(pred_adata.obs.library_id.unique()[0])
    adata_list.append(adata_)


In [ ]:
for i in range(3,7):
    for adata in adata_list:
#         st.em.run_pca(adata,n_comps=50)
        print(i)

        st.tl.clustering.kmeans(adata,n_clusters=i, use_data="X_pca", key_added="X_pca_kmeans")

        st.pl.cluster_plot(adata, use_label="X_pca_kmeans")
        plt.show()

In [ ]:
for adata in adata_list:
    st.pp.scale(adata)
    st.em.run_pca(adata,n_comps=50)

In [ ]:
for i in range(2,6):
    for adata in adata_list:
#         st.em.run_pca(adata,n_comps=50)
        print(i/10)

        st.pp.neighbors(adata,n_neighbors=35,use_rep='X_pca')
        st.tl.clustering.louvain(adata, resolution=i/10)
        st.pl.cluster_plot(adata,use_label="louvain")
        plt.show()

In [ ]:
pred_adata_ = adata_list[0]
test_dataset_ = adata_list[1]

In [ ]:
        st.pp.neighbors(pred_adata_,n_neighbors=35,use_rep='X_pca')
        st.tl.clustering.louvain(pred_adata_, resolution=0.4)
        st.pl.cluster_plot(pred_adata_,use_label="louvain")

In [ ]:
        st.pp.neighbors(test_dataset_,n_neighbors=35,use_rep='X_pca')
        st.tl.clustering.louvain(test_dataset_, resolution=0.2)
        st.pl.cluster_plot(test_dataset_,use_label="louvain")

In [ ]:
for gene in test_dataset_.var_names[0:5]:
    gene_plot(test_dataset_, genes=[gene])

In [ ]:
gene_plot(test_dataset_, genes=["KRT5"])

In [ ]:
test_dataset_

In [ ]:
test_dataset = read_h5ad("/scratch/imb/Xiao/Q1851/Xiao/STimage_dataset/breast_cancer_10x_visium/dataset_2/test_adata.h5ad") 
train_dataset = read_h5ad("/scratch/imb/Xiao/Q1851/Xiao/STimage_dataset/breast_cancer_10x_visium/dataset_2/train_adata.h5ad") 








In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
2518*0.90

In [ ]:
model = CNN_NB_multiple_genes((299, 299, 3), n_genes)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20,
                                            restore_best_weights=False)

In [ ]:
train_history = model.fit(train_gen_,
                          epochs=50,
                          validation_data=valid_gen_,
                          callbacks=[callback]
                          )

In [ ]:
test_predictions = model.predict(test_gen_)
from scipy.stats import nbinom
y_preds = []
for i in range(n_genes):
    n = test_predictions[i][:, 0]
    p = test_predictions[i][:, 1]
    y_pred = nbinom.mean(n, p)
    y_preds.append(y_pred)
test_dataset.obsm["predicted_gene"] = np.array(y_preds).transpose()

In [ ]:
test_dataset_ = test_dataset[:,gene_list].copy()
test_dataset_.X = test_dataset_.obsm["predicted_gene"]

In [ ]:
for i in gene_list:
    print(i)
    gene_plot(test_dataset_, genes=i, spot_size=8)

In [ ]:
gene_plot(test_dataset, genes="COX6C", spot_size=8)

In [ ]:
gene_plot(test_dataset, genes="KRT5", spot_size=8)

In [ ]:
gene_list[6]

In [ ]:
def model_predict_gene(gene):
    i = gene_list.index(gene)
    from scipy.stats import nbinom
    def model_predict(x):
        test_predictions = model.predict(x)
        n = test_predictions[i][:, 0]
        p = test_predictions[i][:, 1]
        y_pred = nbinom.mean(n, p)
        return y_pred.reshape(-1,1)
    return model_predict

In [ ]:
import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
from splot.esda import moran_scatterplot, lisa_cluster
from esda.moran import Moran, Moran_Local
from esda.moran import Moran_BV, Moran_Local_BV
from splot.esda import plot_moran_bv_simulation, plot_moran_bv, plot_local_autocorrelation

In [ ]:
gene = gene_list[4]

In [ ]:
test_dataset_.obsm["gpd"] = gpd.GeoDataFrame(test_dataset_.obs,
                                             geometry=gpd.points_from_xy(
                                                 test_dataset_.obs.imagecol, 
                                                 test_dataset_.obs.imagerow))


In [ ]:
test_dataset_.obsm["gpd"]

In [ ]:
x = test_dataset.to_df()[gene].values
y = test_dataset_.to_df()[gene].values
w = Queen.from_dataframe(test_dataset_.obsm["gpd"])

In [ ]:
test_dataset_.obsm["gpd"]["gc_{}".format(gene)] = x
test_dataset_.obsm["gpd"]["pred_{}".format(gene)] = y
tissue_image = test_dataset_.uns["spatial"]["block2"]["images"]["fulres"]

In [ ]:
moran = Moran(y,w)
moran_bv = Moran_BV(y, x, w)
moran_loc = Moran_Local(y, w)
moran_loc_bv = Moran_Local_BV(y, x, w)

In [ ]:
plot_correlation(test_dataset_.obsm["gpd"],
                 "pred_{}".format(gene),
                 "gc_{}".format(gene))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
moran_scatterplot(moran_bv, ax=ax)
ax.set_xlabel('prediction of gene {}'.format(gene))
ax.set_ylabel('Spatial lag of ground truth of gene {}'.format(gene))
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
moran_scatterplot(moran_loc_bv, p=0.05, ax=ax)
ax.set_xlabel('prediction of gene {}'.format(gene))
ax.set_ylabel('Spatial lag of ground truth of gene {}'.format(gene))
plt.tight_layout()
plt.show()

In [ ]:
def plot_choropleth(gdf, 
                    attribute_1,
                    attribute_2,
                    bg_img,
                    alpha=0.5,
                    scheme='Quantiles', 
                    cmap='YlGnBu', 
                    legend=True):
    
    fig, axs = plt.subplots(2,1, figsize=(5, 8),
                            subplot_kw={'adjustable':'datalim'})
    
    # Choropleth for attribute_1
    gdf.plot(column=attribute_1, scheme=scheme, cmap=cmap,
             legend=legend, legend_kwds={'loc': 'upper left',
                                         'bbox_to_anchor': (0.92, 0.8)},
             ax=axs[0], alpha=alpha, markersize=2)
    
    axs[0].imshow(bg_img)
    axs[0].set_title('choropleth plot for {}'.format(attribute_1), y=0.8)
    axs[0].set_axis_off()
    
    # Choropleth for attribute_2
    gdf.plot(column=attribute_2, scheme=scheme, cmap=cmap,
             legend=legend, legend_kwds={'loc': 'upper left',
                                         'bbox_to_anchor': (0.92, 0.8)},
             ax=axs[1], alpha=alpha, markersize=2)
    
    axs[1].imshow(bg_img)
    axs[1].set_title('choropleth plot for {}'.format(attribute_2), y=0.8)
    axs[1].set_axis_off()
    
    plt.tight_layout()
    
    return fig, ax 

In [ ]:
plot_choropleth(test_dataset_.obsm["gpd"], 
                "gc_{}".format(gene),
                "pred_{}".format(gene),
                tissue_image)
plt.show()

In [ ]:
lisa_cluster(moran_loc_bv, test_dataset_.obsm["gpd"], p=0.05, 
             figsize = (9,9), markersize=12, **{"alpha":0.8})
plt.imshow(test_dataset_.uns["spatial"]["block2"]["images"]["fulres"])
plt.show()

In [ ]:
1